In [1]:
pip install fasttext

StatementMeta(c99f7e44-fcd4-4ee0-8ec8-7fc665560eee, 28, 6, Finished, Available)

Successfully mount artifact container to path:  /synfs/notebook/AmlJobLogs/dcid.18aeb47c-5d25-4c5c-a864-987f8a95eeae
Starting log upload threads
     |████████████████████████████████| 68 kB 8.3 MB/s  eta 0:00:01
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp38-cp38-linux_x86_64.whl size=3324792 sha256=fefb8f6c07397bfe2ee4edd3e433402d06019766ec824a3348fff1ded0c75160
  Stored in directory: /home/trusted-service-user/.cache/pip/wheels/93/61/2a/c54711a91c418ba06ba195b1d78ff24fcaad8592f2a694ac94
Successfully built fasttext
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install emoji

StatementMeta(c99f7e44-fcd4-4ee0-8ec8-7fc665560eee, 28, 7, Finished, Available)

     |████████████████████████████████| 358 kB 9.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

StatementMeta(c99f7e44-fcd4-4ee0-8ec8-7fc665560eee, 28, 8, Finished, Available)

[nltk_data] Downloading package punkt to /home/trusted-service-
[nltk_data]     user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/trusted-service-
[nltk_data]     user/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/trusted-service-user/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /home/trusted-service-
[nltk_data]     user/nltk_data...


True

In [4]:
from tensorflow.keras.models import load_model
model_path='lstm_model.h5'
model = load_model(model_path)

StatementMeta(c99f7e44-fcd4-4ee0-8ec8-7fc665560eee, 28, 9, Finished, Available)

2023-09-27 17:45:59.901789: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
import fasttext
fasttext_model_path='Users/Tanvi.20B0311066/fasttext1.model'
fasttext_model = fasttext.load_model(fasttext_model_path)

StatementMeta(c99f7e44-fcd4-4ee0-8ec8-7fc665560eee, 28, 10, Finished, Available)

In [16]:
import string
import emoji
import re
import nltk
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import fasttext

class NewsClassifier:
    def __init__(self, model, fasttext_model):
        # Load the Keras LSTM model
        self.model = model
        
        # Load the FastText model
        self.fasttext_model = fasttext_model

    def preprocess_text(self, text):
        # Lowercasing
        text = text.lower()

        # Removing punctuations using string module
        text = text.translate(str.maketrans("", "", string.punctuation))

        # Removing URLs and Emojis
        text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)
        text = emoji.demojize(text)
        text = re.sub(r":[^:]+:", "", text)

        # Removing Emails and Numbers
        email_pattern = r'\S+@\S+\.\S+'
        number_pattern = r'\b\d+\b'
        text = re.sub(email_pattern, '', text)
        text = re.sub(number_pattern, '', text)

        # Removing Special Characters
        pattern = r'[^a-zA-Z\s]'
        text = re.sub(pattern, '', text)
        text = text.replace('\n', ' ')
        text = text.encode('ascii', 'ignore').decode('ascii')  # Remove non-ASCII characters

        # Tokenization
        tokens = word_tokenize(text)

        # Removing stopwords
        stops = set(stopwords.words('english'))
        tokens = [token for token in tokens if token.lower() not in stops]

        # Lemmatization
        lemmatizer = WordNetLemmatizer()
        def pos_tagger(nltk_tag):
            if nltk_tag.startswith('J'):
                return wordnet.ADJ
            elif nltk_tag.startswith('V'):
                return wordnet.VERB
            elif nltk_tag.startswith('N'):
                return wordnet.NOUN
            elif nltk_tag.startswith('R'):
                return wordnet.ADV
            else:
                return wordnet.NOUN  # Default to noun
        pos_tagged = nltk.pos_tag(tokens)
        wordnet_tagged = [(word, pos_tagger(tag)) for word, tag in pos_tagged]
        lemmatized = [lemmatizer.lemmatize(word, tag) for word, tag in wordnet_tagged]

        return ' '.join(lemmatized)

    def fasttext_vectorize(self, text):
        # Tokenization
        tokens = word_tokenize(text)

        embeddings = []
        for token in tokens:
            if token in self.fasttext_model.words:
                embeddings.append(self.fasttext_model.get_word_vector(token))
        
        if embeddings:
            return sum(embeddings) / len(embeddings)
        return None

    def predict_category(self, text):
        preprocessed_text = self.preprocess_text(text)
        text_vectorized = self.fasttext_vectorize(preprocessed_text)
        text_vectorized=text_vectorized.reshape(1,300)
        if text_vectorized is not None:
            predictions = self.model.predict(np.array([text_vectorized]))
            predicted_class = np.argmax(predictions[0])
            return predicted_class
        else:
            return "Unable to vectorize input text"


StatementMeta(c99f7e44-fcd4-4ee0-8ec8-7fc665560eee, 28, 21, Finished, Available)

In [32]:
input_text = '''BJP'''

StatementMeta(c99f7e44-fcd4-4ee0-8ec8-7fc665560eee, 28, 37, Finished, Available)

In [33]:
classifier = NewsClassifier(model,fasttext_model)

StatementMeta(c99f7e44-fcd4-4ee0-8ec8-7fc665560eee, 28, 38, Finished, Available)

In [34]:
import time
start_time = time.time()
predicted_index= classifier.predict_category(input_text)
end_time = time.time()
inference_time = end_time - start_time
category_labels = [
    'academic interests', 'arts and culture', 'automotives', 'books and literature',
    'business and finance', 'careers', 'family and relationships', 'food and drinks',
    'health', 'healthy living', 'hobbies and interests', 'home and garden', 'movies',
    'music and audio', 'news and politics', 'personal finance', 'pets',
    'pharmaceuticals, conditions, and symptoms', 'real estate', 'shopping', 'sports',
    'style and fashion', 'technology and computing', 'television', 'travel', 'video gaming'
]
predicted_label = category_labels[predicted_index]
print(f"Predicted category: {predicted_label}")
print("INFERENCE TIME USING LSTM:- ",inference_time)

StatementMeta(c99f7e44-fcd4-4ee0-8ec8-7fc665560eee, 28, 39, Finished, Available)

Predicted category: news and politics
INFERENCE TIME USING LSTM:-  0.033579349517822266
